In [2]:
]box on
]rows on

Was ON

Was OFF

Let's start by parsing the input data. This is the magic incantation for reading a file line-by-line. You can see that we get an array of strings, where some of the strings are empty.

In [29]:
data←⊃⎕NGET'input.txt'1
data

┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬┬────┬────┬──
│3427│3273│5615│5943│3125│4245│4194│3243│4283│1790│5355│4239│5541││3850│5294│53
└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴┴────┴────┴──
 ──┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬┬────┬────
 93│2490│3231│2743│2016│2418│2304│5673│3035│5292│2112│2630│2305││2325│2002
 ──┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴┴────┴────
 ┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬┬────┬────┬─
 │3402│4756│1662│1133│5273│6397│2977│4011│3594│4441│3907│1503││7598│6971│6
 ┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴┴────┴────┴─
 ───┬────┬────┬────┬────┬────┬────┬────┬┬─────┬────┬─────┬┬────┬────┬────┬
 979│3873│1394│8399│1321│2304│2570│6679││15324│3365│23808││5489│3526│3125│
 ───┴────┴────┴────┴────┴────┴────┴────┴┴─────┴────┴─────┴┴────┴────┴────┴
 ────┬────┬────┬────┬─────┬┬────┬────┬────┬────┬────┬────┬────┬────┬────┬─
 8371│1928│8191│7267│10728││5243│2203│7331│8049│4951│3743│2255│4112│3077│2
 ────┴────┴────┴────┴─────┴┴────┴────┴────┴────┴────┴────┴────┴────┴────┴─
 ───┬────┬┬─────┬────┬─────┬─────┬─────┬────┬┬────┬────┬────┬────┬────┬───
 441│4678││12353│2251│12157│13656│10655│7525││3383│4183│3293│5473│4898│144
 ───┴────┴┴─────┴────┴─────┴─────┴─────┴────┴┴────┴────┴────┴────┴────┴───
 ─┬────┬────┬────┬────┬────┬────┬────┬────┬┬────┬────┬────┬────┬────┬────┬
 1│3786│4002│2179│2439│5530│3966│6273│3150││3320│2694│2229│1394│4568│3160│
 ─┴────┴────┴────┴────┴────┴────┴────┴────┴┴────┴────┴────┴────┴────┴────┴
 ────┬────┬────┬────┬────┬────┬────┬────┬────┬┬─────┬─────┬┬─────┬────┬───
 4986│2487│3412│1030│3717│1460│1140│1314│5816││15303│36095││14345│5208│225
 ────┴────┴────┴────┴────┴────┴────┴────┴────┴┴─────┴─────┴┴─────┴────┴───
 ─┬─────┬┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────
 3│19532││5082│6544│3563│6050│3750│1155│2090│3661│3600│5528│6506│1795│4901
 ─┴─────┴┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────
 ┬┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬┬────┬────┬────┬
 ││1156│1867│2025│5434│1546│6052│4412│7203│4622│7098│4727││4423│1829│4761│
 ┴┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴┴────┴────┴────┴
 ────┬────┬────┬────┬────┬────┬────┬────┬┬────┬────┬────┬─────┬────┬────┬─
 4779│3234│4869│3567│6928│4181│5390│5938││3100│6087│9235│10228│2082│5472│1
 ────┴────┴────┴────┴────┴────┴────┴────┴┴────┴────┴────┴─────┴────┴────┴─
 ────┬┬────┬─────┬─────┬────┬────┬┬────┬────┬────┬────┬────┬────┬────┬────
 1382││8389│15131│12057│5034│9419││2192│1695│3233│2245│4328│6275│1811│4345
 ────┴┴────┴─────┴─────┴────┴────┴┴────┴────┴────┴────┴────┴────┴────┴────
 ┬────┬────┬────┬────┬────┬────┬┬────┬────┬────┬────┬────┬────┬────┬────┬─
 │2760│3205│1138│2795│3876│1041││1302│3509│6074│1793│5808│5364│3802│2048│6
 ┴────┴────┴────┴────┴────┴────┴┴────┴────┴────┴────┴────┴────┴────┴────┴─
 ───┬────┬────┬────┬────┬────┬────┬┬────┬────┬────┬────┬────┬────┬────┬───
 042│4913│5983│1189│3134│3763│2573││1003│8411│2571│2555│7555│8096│1441│560
 ───┴────┴────┴────┴────┴────┴────┴┴────┴────┴────┴────┴────┴────┴────┴───
 ─┬────┬────┬┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬
 0│5104│6311││5345│3329│5616│3250│4321│2068│1978│3719│4512│5099│4694│1566│
 ─┴────┴────┴┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴
 ────┬────┬┬─────┬────┬────┬────┬────┬────┬────┬┬────┬────┬────┬────┬────┬
 2621│1613││11021│6101│9858│3142│9907│1578│8582││3182│7695│4419│4036│5846│
 ────┴────┴┴─────┴────┴────┴────┴────┴────┴────┴┴────┴────┴────┴────┴────┴
 ────┬────┬────┬────┬────┬┬────┬────┬─────┬────┬────┬────┬┬────┬────┬────┬
 3899│6104│4495│7382│3471││6844│3290│10505│7855│1494│5963││3609│4755│4063│
 ────┴────┴────┴────┴────┴┴────┴────┴─────┴────┴────┴────┴┴────┴────┴────┴
 ────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬┬────┬────┬────┬──
 1480│3201│4576│1627│1255│3699│3929│5533│4807│5890│2270││3789│7045│8784│80
 ────┴────

Now let's split it up into groups separated by those empty elements. To do that, we're using partition (⊆), which takes our data on the right, and a binary vector on the right. Everywhere where the binary vector is 0, we'll get a split. For example:

In [17]:
1 1 0 1 0 1 1⊆⍳7

┌───┬─┬───┐
│1 2│4│6 7│
└───┴─┴───┘

To know where to split the data, we need 1s where there's a non-empty string, and 0s where there's not. Tally (≢) gives us the length of a vector (strings are just character vectors), and direction (×) turns positive numbers into 1s and leaves 0s alone. (It also turns negative numbers into -1s, but that's not important right now.)

So all we need to do is map ≢ over the data, then apply × to the whole thing:

In [32]:
×≢¨data

1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 
 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 
 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 
 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 
 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 0 
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 
 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 
 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 
 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 
 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 
 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 
 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 
 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 
 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1

That gives us the binary vector we need for the left argument to partition (⊆). 

A quick note: I'm mixing (↑) the results below to make them more readable. It just turns arrays of arrays into matrices.

We could write it like this:

In [34]:
↑(×≢¨data)⊆data

┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬
│3427 │3273 │5615 │5943 │3125 │4245 │4194 │3243 │4283 │1790 │5355 │4239 │5541 │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│3850 │5294 │5393 │2490 │3231 │2743 │2016 │2418 │2304 │5673 │3035 │5292 │2112 │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│2325 │2002 │3402 │4756 │1662 │1133 │5273 │6397 │2977 │4011 │3594 │4441 │3907 │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│7598 │6971 │6979 │3873 │1394 │8399 │1321 │2304 │2570 │6679 │ │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│15324│3365 │23808│ │ │ │ │ │ │ │ │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│5489 │3526 │3125 │8371 │1928 │8191 │7267 │10728│ │ │ │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│5243 │2203 │7331 │8049 │4951 │3743 │2255 │4112 │3077 │2441 │4678 │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│12353│2251 │12157│13656│10655│7525 │ │ │ │ │ │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│3383 │4183 │3293 │5473 │4898 │1441 │3786 │4002 │2179 │2439 │5530 │3966 │6273 │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│3320 │2694 │2229 │1394 │4568 │3160 │4986 │2487 │3412 │1030 │3717 │1460 │1140 │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│15303│36095│ │ │ │ │ │ │ │ │ │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│14345│5208 │2253 │19532│ │ │ │ │ │ │ │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│5082 │6544 │3563 │6050 │3750 │1155 │2090 │3661 │3600 │5528 │6506 │1795 │4901 │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│1156 │1867 │2025 │5434 │1546 │6052 │4412 │7203 │4622 │7098 │4727 │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│4423 │1829 │4761 │4779 │3234 │4869 │3567 │6928 │4181 │5390 │5938 │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│3100 │6087 │9235 │10228│2082 │5472 │11382│ │ │ │ │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│8389 │15131│12057│5034 │9419 │ │ │ │ │ │ │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│2192 │1695 │3233 │2245 │4328 │6275 │1811 │4345 │2760 │3205 │1138 │2795 │3876 │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│1302 │3509 │6074 │1793 │5808 │5364 │3802 │2048 │6042 │4913 │5983 │1189 │3134 │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│1003 │8411 │2571 │2555 │7555 │8096 │1441 │5600 │5104 │6311 │ │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│5345 │3329 │5616 │3250 │4321 │2068 │1978 │3719 │4512 │5099 │4694 │1566 │2621 │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│11021│6101 │9858 │3142 │9907 │1578 │8582 │ │ │ │ │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│3182 │7695 │4419 │4036 │5846 │3899 │6104 │4495 │7382 │3471 │ │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│6844 │3290 │10505│7855 │1494 │5963 │ │ │ │ │ │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│3609 │4755 │4063 │1480 │3201 │4576 │1627 │1255 │3699 │3929 │5533 │4807 │5890 │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│3789 │7045 │8784 │8006 │4583 │7726 │3993 │1472 │1578 │2046 │ │ │ │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼
│2474 │5953 │2050 │4758 │5265 │5075 │4338 │1340 │1301 │5996 │1225 │1794 │4778 │
├─────┼─────┼─────┼─────┼───

But anytime you need a value on both sides of an operator, you can instead use a fork. A fork follows this form:

`(f x) h (g x) = (f h g) x`

In my case, I have `(×≢¨data)⊆data = (×≢¨)⊆⊢)data`. The ⊢ is just the identity function, because I don't want to do anything to `data` in the right-hand argument. If I leave the data out, you can see the structure of the fork:

In [35]:
((×≢¨)⊆⊢)

┌──┼─┐
┌┴┐ ⊆ ⊢
× ¨
┌─┘
≢

In [42]:
((×≢¨)⊆⊢)data

┌──────────────────────────────────────────────────────────────────┬───────────
│┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐│┌────┬────┬
││3427│3273│5615│5943│3125│4245│4194│3243│4283│1790│5355│4239│5541│││3850│5294│
│└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘│└────┴────┴
└──────────────────────────────────────────────────────────────────┴───────────
 ─────────────────────────────────────────────────────────────────┬───────
 ────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐│┌────┬─
 5393│2490│3231│2743│2016│2418│2304│5673│3035│5292│2112│2630│2305│││2325│2
 ────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘│└────┴─
 ─────────────────────────────────────────────────────────────────┴───────
 ────────────────────────────────────────────────────────────────┬────────
 ───┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐│┌────┬──
 002│3402│4756│1662│1133│5273│6397│2977│4011│3594│4441│3907│1503│││7598│69
 ───┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘│└────┴──
 ────────────────────────────────────────────────────────────────┴────────
 ───────────────────────────────────────────┬──────────────────┬──────────
 ──┬────┬────┬────┬────┬────┬────┬────┬────┐│┌─────┬────┬─────┐│┌────┬────
 71│6979│3873│1394│8399│1321│2304│2570│6679│││15324│3365│23808│││5489│3526
 ──┴────┴────┴────┴────┴────┴────┴────┴────┘│└─────┴────┴─────┘│└────┴────
 ───────────────────────────────────────────┴──────────────────┴──────────
 ────────────────────────────────┬────────────────────────────────────────
 ┬────┬────┬────┬────┬────┬─────┐│┌────┬────┬────┬────┬────┬────┬────┬────
 │3125│8371│1928│8191│7267│10728│││5243│2203│7331│8049│4951│3743│2255│4112
 ┴────┴────┴────┴────┴────┴─────┘│└────┴────┴────┴────┴────┴────┴────┴────
 ────────────────────────────────┴────────────────────────────────────────
 ────────────────┬───────────────────────────────────┬────────────────────
 ┬────┬────┬────┐│┌─────┬────┬─────┬─────┬─────┬────┐│┌────┬────┬────┬────
 │3077│2441│4678│││12353│2251│12157│13656│10655│7525│││3383│4183│3293│5473
 ┴────┴────┴────┘│└─────┴────┴─────┴─────┴─────┴────┘│└────┴────┴────┴────
 ────────────────┴───────────────────────────────────┴────────────────────
 ───────────────────────────────────────────────────┬─────────────────────
 ┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐│┌────┬────┬────┬────┬
 │4898│1441│3786│4002│2179│2439│5530│3966│6273│3150│││3320│2694│2229│1394│
 ┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘│└────┴────┴────┴────┴
 ───────────────────────────────────────────────────┴─────────────────────
 ───────────────────────────────────────────────────────┬─────────────┬───
 ────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐│┌─────┬─────┐│┌──
 4568│3160│4986│2487│3412│1030│3717│1460│1140│1314│5816│││15303│36095│││14
 ────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘│└─────┴─────┘│└──
 ───────────────────────────────────────────────────────┴─────────────┴───
 ────────────────────┬────────────────────────────────────────────────────
 ───┬────┬────┬─────┐│┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬─
 345│5208│2253│19532│││5082│6544│3563│6050│3750│1155│2090│3661│3600│5528│6
 ───┴────┴────┴─────┘│└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴─
 ────────────────────┴────────────────────────────────────────────────────
 ──────────────┬────────────────────────────────────────────────────────┬─
 ───┬────┬────┐│┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐│┌
 506│1795│4901│││1156│1867│2025│5434│1546│6052│4412│7203│4622│7098│4727│││
 ───┴────┴────┘│└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘│└
 ──────────────┴────────────────────────────────────────────────────────┴─
 ───────────────────────────────────────────────────────┬─────────────────
 ────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐│┌────┬────┬────┬─
 4423│1829│4761│4779│3234│4869│3567│6928│4181│5390│5938│││3100│6087│9235│1


Finally, to turn the strings into numbers, I use execute (⍎). Notice I have to map (¨) twice because my values are two arrays deep:

In [43]:
parsed←⍎¨¨((×≢¨)⊆⊢)data
parsed

┌────────────────────────────────────────────────────────────────┬─────────────
│3427 3273 5615 5943 3125 4245 4194 3243 4283 1790 5355 4239 5541│3850 5294 539
└────────────────────────────────────────────────────────────────┴─────────────
 ─────────────────────────────────────────────────────────────┬───────────
 3 2490 3231 2743 2016 2418 2304 5673 3035 5292 2112 2630 2305│2325 2002 3
 ─────────────────────────────────────────────────────────────┴───────────
 ──────────────────────────────────────────────────────────┬──────────────
 402 4756 1662 1133 5273 6397 2977 4011 3594 4441 3907 1503│7598 6971 6979
 ──────────────────────────────────────────────────────────┴──────────────
 ───────────────────────────────────┬────────────────┬────────────────────
 3873 1394 8399 1321 2304 2570 6679│15324 3365 23808│5489 3526 3125 8371 
 ───────────────────────────────────┴────────────────┴────────────────────
 ────────────────────┬────────────────────────────────────────────────────
 1928 8191 7267 10728│5243 2203 7331 8049 4951 3743 2255 4112 3077 2441 46
 ────────────────────┴────────────────────────────────────────────────────
 ──┬─────────────────────────────────┬────────────────────────────────────
 78│12353 2251 12157 13656 10655 7525│3383 4183 3293 5473 4898 1441 3786 4
 ──┴─────────────────────────────────┴────────────────────────────────────
 ─────────────────────────────────┬───────────────────────────────────────
 002 2179 2439 5530 3966 6273 3150│3320 2694 2229 1394 4568 3160 4986 2487
 ─────────────────────────────────┴───────────────────────────────────────
 ───────────────────────────────────┬───────────┬─────────────────────┬───
 3412 1030 3717 1460 1140 1314 5816│15303 36095│14345 5208 2253 19532│508
 ───────────────────────────────────┴───────────┴─────────────────────┴───
 ─────────────────────────────────────────────────────────────┬───────────
 2 6544 3563 6050 3750 1155 2090 3661 3600 5528 6506 1795 4901│1156 1867 2
 ─────────────────────────────────────────────────────────────┴───────────
 ───────────────────────────────────────────┬─────────────────────────────
 025 5434 1546 6052 4412 7203 4622 7098 4727│4423 1829 4761 4779 3234 4869
 ───────────────────────────────────────────┴─────────────────────────────
 ─────────────────────────┬────────────────────────────────────┬──────────
 3567 6928 4181 5390 5938│3100 6087 9235 10228 2082 5472 11382│8389 15131
 ─────────────────────────┴────────────────────────────────────┴──────────
 ────────────────┬────────────────────────────────────────────────────────
 12057 5034 9419│2192 1695 3233 2245 4328 6275 1811 4345 2760 3205 1138 2
 ────────────────┴────────────────────────────────────────────────────────
 ─────────────┬───────────────────────────────────────────────────────────
 795 3876 1041│1302 3509 6074 1793 5808 5364 3802 2048 6042 4913 5983 1189
 ─────────────┴───────────────────────────────────────────────────────────
 ───────────────┬─────────────────────────────────────────────────┬───────
 3134 3763 2573│1003 8411 2571 2555 7555 8096 1441 5600 5104 6311│5345 33
 ───────────────┴─────────────────────────────────────────────────┴───────
 ──────────────────────────────────────────────────────────────┬──────────
 29 5616 3250 4321 2068 1978 3719 4512 5099 4694 1566 2621 1613│11021 6101
 ──────────────────────────────────────────────────────────────┴──────────
 ─────────────────────────┬───────────────────────────────────────────────
 9858 3142 9907 1578 8582│3182 7695 4419 4036 5846 3899 6104 4495 7382 34
 ─────────────────────────┴───────────────────────────────────────────────
 ──┬──────────────────────────────┬───────────────────────────────────────
 71│6844 3290 10505 7855 1494 5963│3609 4755 4063 1480 3201 4576 1627 1255
 ──┴──────────────────────────────┴───────────────────────────────────────
 ──────────────────────────────┬──────────────────────────────────────────
 3699 3929 5533 4807 5890 2270│3789 7045 8784 8006 4583 7726 3993 1472 15
 ────────────────

The rest is easy! For part one, we need to find the subarray with the greatest sum. Let's first sum the subarrays. We can add two things with +, so to sum the array, we just need to reduce (/) using +.

In [47]:
+/¨parsed

54273 50786 47383 48088 42497 48625 48083 58597 53996 42727 51398 41338 54225 4
 6142 49899 47586 50030 40939 57297 48647 49731 50189 50529 35951 50694 49
 022 51520 48710 47711 50880 50932 57041 36461 52012 46026 42166 56362 441
 54 52858 43013 46279 47750 54441 50938 49829 67045 35676 41928 42135 3843
 0 28458 50836 55269 47657 33259 64481 43964 42605 50625 42991 12076 54167
 54143 29731 48538 49039 45914 41817 44333 20512 43194 53755 44543 53998 
 58153 37926 18413 51987 57622 28838 53017 44823 49487 38667 50949 38592 4
 5725 44005 44456 38028 53754 37528 52792 45595 37392 52837 46171 41738 50
 647 57833 53616 55366 59931 60528 42653 9550 39835 48717 50245 49173 6676
 5 41745 43846 38332 47675 38649 43142 55751 53464 50419 35056 17614 50957
 49752 43907 47179 27104 45480 54857 52139 63049 44082 61016 57343 57625 
 48260 33063 56522 23489 46905 48411 40309 54049 44437 46460 52633 58065 5
 1808 27342 53397 48377 39637 41673 51217 46765 43864 61387 63101 48567 64
 498 50773 46496 54351 56821 46047 61230 56446 51690 57835 58222 49340 517
 90 31720 63207 42159 49344 59704 47917 46544 31639 57523 57018 45319 2871
 8 42963 47186 53933 49135 68775 47615 59348 38013 46392 58198 43383 54879
 42776 65924 17079 42302 41116 57210 53940 52994 53876 41680 45182 34591 
 48853 53887 62125 46777 46930 43560 54413 58102 51525 27766 36690 48819 5
 8355 44600 27881 37645 53367 51807 44267 49539 55776 41076 55107 39786 49
 150 47304 54684

Finally, we take the biggest one using maximum (⌈). When used with left and right arguments (dyadically), it returns the larger of the two arguments. Used with reduce (/), it returns the maximum value in an array:

In [48]:
⌈/+/¨parsed

68775

And that's the answer to part one!

For part two, we need the sum of the greatest 3 subarrays. That means we need to sort the data in descending order. I'm not going to go into the details of the sorting incantation, but [you can read more about it here](https://xpqz.github.io/learnapl/manip.html?highlight=sort#grade-up-down).

In [74]:
(⊂∘⍒⌷⊢)+/¨parsed

68775 67045 66765 65924 64498 64481 63207 63101 63049 62125 61387 61230 61016 6
 0528 59931 59704 59348 58597 58355 58222 58198 58153 58102 58065 57835 57
 833 57625 57622 57523 57343 57297 57210 57041 57018 56821 56522 56446 563
 62 55776 55751 55366 55269 55107 54879 54857 54684 54441 54413 54351 5427
 3 54225 54167 54143 54049 53998 53996 53940 53933 53887 53876 53755 53754
 53616 53464 53397 53367 53017 52994 52858 52837 52792 52633 52139 52012 
 51987 51808 51807 51790 51690 51525 51520 51398 51217 50957 50949 50938 5
 0932 50880 50836 50786 50773 50694 50647 50625 50529 50419 50245 50189 50
 030 49899 49829 49752 49731 49539 49487 49344 49340 49173 49150 49135 490
 39 49022 48853 48819 48717 48710 48647 48625 48567 48538 48411 48377 4826
 0 48088 48083 47917 47750 47711 47675 47657 47615 47586 47383 47304 47186
 47179 46930 46905 46777 46765 46544 46496 46460 46392 46279 46171 46142 
 46047 46026 45914 45725 45595 45480 45319 45182 44823 44600 44543 44456 4
 4437 44333 44267 44154 44082 44005 43964 43907 43864 43846 43560 43383 43
 194 43142 43013 42991 42963 42776 42727 42653 42605 42497 42302 42166 421
 59 42135 41928 41817 41745 41738 41680 41673 41338 41116 41076 40939 4030
 9 39835 39786 39637 38667 38649 38592 38430 38332 38028 38013 37926 37645
 37528 37392 36690 36461 35951 35676 35056 34591 33259 33063 31720 31639 
 29731 28838 28718 28458 27881 27766 27342 27104 23489 20512 18413 17614 1
 7079 12076 9550

Then we just take the first 3 and sum-reduce. When used dyadically, ↑ means "take":

In [75]:
3↑(⊂∘⍒⌷⊢)+/¨parsed

68775 67045 66765

In [76]:
+/3↑(⊂∘⍒⌷⊢)+/¨parsed

202585

And that's our answer for part two!